# Basic Read Operations

Design several subroutines to handle different read operations:

- Handle strings
- Handle numbers
- Handle in-sheet cell references
- Handle out-of-sheet references
- Handle cell range references

In [15]:
from openpyxl.formula import Tokenizer

from openpyxl.utils import quote_sheetname, absolute_coordinate

class ExcelParser():
    """
    Perform the basic operations required to interpret
    the contents of an excel cell, then output a 
    pythonic equivalent.

    Note: A pythonic equivalent is not the only
    solutionm non-python code could also be generated
    """
    
    def __init__(self,workbook):
        self.varDict = {}
        self.calculableDict={}
        self.definedNames={}
        self.get_defined_names(workbook)
        
    def get_defined_names(self,workbook):
        """
        Aquire all global defined names
        for comparison and replacement purposes 
        """
        for definedName in workbook.defined_names.definedName:
            
            cellName = str(definedName.attr_text).split("!")
            
            cellLocation = self.variable_name(cellName[0],cellName[1]) 
            
            self.definedNames[f'{definedName.name}'] = f'{cellLocation}'
            
        try:
            for sheet in workbook:
                for definedName in sheet.defined_names:
                    print(definedName)
        except:
            pass
        

    def first_read(self,sheet,cell):
        """ 
        Perform first read of cell to establish
        what functions need to be called to process the cell.
        """


        # discard empty cell

        if type(cell.value) == type(None):
            return

        # check if cell is calculable 
        # if not, assume it is a variable and allocate

        if not self.is_calculable(cell):
            self.create_variable(sheet,cell)
            return
        else:
            self.function_parse(sheet,cell)
        


    def is_calculable(self,cell):
        """
        Checks if the cell begins with an '='
        and needs calculating
        """
        value = str(cell.value)
        if value[0] == '=':
            return True
        else:
            return False

        
    def variable_name(self,sheet,coordinate):
        """
        Automatically generate the correct
        variable name
        """
        
        sheetName=str(sheet).strip()
        sheetName=sheetName.replace(" ","")
        coordinate=coordinate.replace("$","")
        
        return f'{sheetName}_{coordinate}'
    

    def create_variable(self,sheet,cell):
        """
        Given a cell is not calculable, we assume it
        is a variable. Create a unique variable name and
        allocate the value. Record all variables.

        Variables should be of the form:
        sheetName-cellName : mysheet1-B1
        or
        definedName: myDefinedCallName
        """
      
        
        varName = self.variable_name(sheet,cell.coordinate)
        if varName in self.definedNames.values():
            for key,value in self.definedNames.items():
                if(varName == value):
                    varName = key
                    break
        
        if varName in self.varDict:
            
            raise ValueError(f"""Variable name '{varName}' already exists.\n
                            Convention uses sheetName-cellName. 
                            If this is raised check if name appears more than once.""")
        
        
        self.varDict[varName] = f'{cell.value}'
        
    
    def function_parse(self,sheet,cell):
        """
        Determine the type of functions used
        and split into simple functions, semi-complex
        and complex functions.
        
        Simple functions:
        Standard operands (+,-,*,/,powers)
        
        Semi-complex functions:
        Anything that can be used with a simple switch out
        e.g SQRT() -> math.sqrt() in python, or sqrt() in C
        
        Complex:
        Functions that require near explicit programming to 
        replicate the required functionality
        e.g SUM() -> bespoke summing function

        """
        
        # tokenise the function to be able to inspect the functions
        tokenString = Tokenizer(str(cell.value))
        
        isComplexFunc = False
        
        # Determine if mathematical functions are semi/complex
        for token in tokenString.items:
            if token.type  == 'FUNC':
                isComplexFunc = True
                break
                
        if not isComplexFunc:
            self.simple_function(sheet, cell,tokenString)
        else:
            self.complex_function(sheet, cell)



    def simple_function(self,sheet,cell,tokenString):
        """
        Parse the simple functions. 
        Parsing should:
        - interpret the mathematics and output function code
        - replace cell coordinates with cell variable names
        """
        functionString =""
        functionVars =[]
        for token in tokenString.items:
            val =""
            if token.subtype == 'RANGE':
                if '!' in token.value:
                    val =str(token.value).strip()
                    val=val.replace(" ","")
                    val=val.replace("'","")
                    val=val.replace('!','_')
                elif str(token.value) in self.definedNames:
                    val = str(token.value)
                else:
                    val=self.variable_name(sheet,token.value)
                
                functionVars.append(val)    
            else:
                val=token.value
            
            functionString+=val
            
        self.calculableDict[self.variable_name(sheet,cell.coordinate)] = [functionString,functionVars]
        
        
    def complex_function(self,sheet,cell):
        print("Complex")

## Output code
### Language
Allow the user to select the language to be output to.

Initial language output is python, but support for R and C is expected

### Program construction

The excel workbook is cycled from sheet 0 to sheet N-1, top left corner to bottom right of each sheet. While this allows us to grab all the variables, and corresponding functions, it does not give us correct order that the variables and functions need to appear in a program.

For example, given the following excel input:

- var1 = 1
- var2 = 3
- var3 = var1+var2+var4
- var4

We see that in excel this does not matter. Each cell cascades until all values can be resolved.

However, in programming languages the calculation of var3 will cause the compile/interpreter to refuse to run, as var4 has not been initialised prior to the variable call. 

This requires that each time a calculable cell is called, the variables for the cell need to be initialised prior. To achieve this we can construct the program from a calculable cell point of view. 

- First we separate the variables and the calculable cells into two categories. 
- We then initialise all the variables.
- We then step through the calculable cells and check if all variables for it exist. 
- If not, we assume that the cell relies on a prior calculable cell, and so move to the next cell.
- Loop through all calculable cells until all cells have been added to the program.
- To print values we will rely on user guidance, i.e we will request which cells should be output and then have functions to handle the outputs





In [16]:
#class ProgramConversion():
#    """
#    Class to handle the conversion of Excel in
#    a new language.
#    Current supported languages:
#    - Python
#    """
#    
#    def __init__(self,language, outputFile):
#        self.language = language
#        self.languageFileTypes = {'python':'.py'}
#        self.output = outputFile
#        
#    def write_out_code(self, codeString):
#        """
#        Open code file and print code to 
#        file.
#        """
#        
#        with open(f'{self.output}{self.languageFileTypes[self.language]}') as f:
#            f.write(codeString)
            
    
class PythonConvert():
    """
    Write python code from Excel conversion
    """
    
    def __init__(self,varDict,calculableDict):
        
        self.codeString = ""
        self.formulaeOrder = {}
        self.varsInCode=[]
        self.varDict = varDict
        self.calculableDict = calculableDict
        self.vars_to_code()
        self.calculable_to_code()

    
    
            
    def vars_to_code(self):
        """
        Write out all vars to code string
        """
        self.codeString+="""# Defined variables that are not changable inputs\n# Each variable comes from a single cell in the excel sheet\n# and is named as Sheet_CellCoordinates\n\n"""
        for var in self.varDict.items():
            self.varsInCode.append(var[0])
            self.codeString+=f"{var[0]} = {var[1]}\n\n"
            
    def calculable_to_code(self):
        """
        Write out functions into code, but first checking if required
        vars are already written
        """
        self.codeString+="""# Variables that require being calculated before use\n# Each variable comes from a single cell in the excel sheet\n# and is named as Sheet_CellCoordinates\n\n"""
        
        tempDict = self.calculableDict
        count = 0
        while(len(tempDict)>0 and count <=100):
            for key in list(tempDict.keys()): 
                canAdd = True
                for neededVar in tempDict[key][1]:

                    if neededVar not in self.varsInCode:
                        canAdd = False
                        #print("")
                
                if canAdd==True:
                    self.formulaeOrder[key] = tempDict[key]
                    self.codeString+=f"{key} = {tempDict[key][0]}\n\n"
                    #print(f"{key} = {tempDict[key][0]}\n\n")
                    self.varsInCode.append(key)
                    del tempDict[key]
                    
            count+=1
        
        if(tempDict):
            print("-------------------------Unresolved functions--------------------------")
            print(tempDict)
            print("-------------------------Unresolved functions--------------------------")
            
            
       
        
        
        
        
        

In [19]:
import openpyxl
import os

fileNameAndPath="C:/Users/mo14776/OneDrive - University of Bristol/Documents/Health Economics/REEVER/Examples/Tests for the Excel- R conversion/test_workbook_2.xlsx"

excelBook = openpyxl.load_workbook(fileNameAndPath, keep_vba=True)

excelParser = ExcelParser(excelBook)

for sheet in excelBook.sheetnames:
    for row in excelBook[sheet].iter_rows():
        for cell in row:
            
            excelParser.first_read(sheet,cell)
            
pythoncode = PythonConvert(excelParser.varDict,excelParser.calculableDict)

print(pythoncode.codeString)

# Defined variables that are not changable inputs
# Each variable comes from a single cell in the excel sheet
# and is named as Sheet_CellCoordinates

Frontend_D4 = Inputs

Frontend_D5 = Willingness-to-pay

willingness_to_pay = 20000

Frontend_D6 = Outputs

Frontend_D7 = ICER Warfarin vs aspirin

Frontend_D8 = INB Warfarin vs aspirin

Inputparameters_D3 = Name

Inputparameters_E3 = Value

Inputparameters_D4 = Probability stroke aspirin

Inputparameters_E4 = 0.1

Inputparameters_D5 = Probability stroke warfarin

Inputparameters_E5 = 0.05

Inputparameters_D6 = Probability bleed aspirin

Inputparameters_E6 = 0.025

Inputparameters_D7 = Probability bleed warfarin

Inputparameters_E7 = 0.05

Inputparameters_D8 = Cost course aspirin

Inputparameters_E8 = 10

Inputparameters_D9 = Cost course warfarin

Inputparameters_E9 = 500

Inputparameters_D10 = Cost stroke

Inputparameters_E10 = 800

Inputparameters_D11 = Cost bleed

Inputparameters_E11 = 400

Inputparameters_D12 = QALYS stroke

Inputpara

In [18]:
print(excelParser.varDict) 
print(excelParser.calculableDict)

{'Frontend_D4': 'Inputs', 'Frontend_D5': 'Willingness-to-pay', 'willingness_to_pay': '20000', 'Frontend_D6': 'Outputs', 'Frontend_D7': 'ICER Warfarin vs aspirin', 'Frontend_D8': 'INB Warfarin vs aspirin', 'Inputparameters_D3': 'Name', 'Inputparameters_E3': 'Value', 'Inputparameters_D4': 'Probability stroke aspirin', 'Inputparameters_E4': '0.1', 'Inputparameters_D5': 'Probability stroke warfarin', 'Inputparameters_E5': '0.05', 'Inputparameters_D6': 'Probability bleed aspirin', 'Inputparameters_E6': '0.025', 'Inputparameters_D7': 'Probability bleed warfarin', 'Inputparameters_E7': '0.05', 'Inputparameters_D8': 'Cost course aspirin', 'Inputparameters_E8': '10', 'Inputparameters_D9': 'Cost course warfarin', 'Inputparameters_E9': '500', 'Inputparameters_D10': 'Cost stroke', 'Inputparameters_E10': '800', 'Inputparameters_D11': 'Cost bleed', 'Inputparameters_E11': '400', 'Inputparameters_D12': 'QALYS stroke', 'Inputparameters_E12': '14', 'Inputparameters_D13': 'QALYS bleed', 'Inputparameters_